# MIDI Tempo Detective Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

Credit for GPT2-RGA code used in this colab goes out @ Sashmark97 https://github.com/Sashmark97/midigen and @ Damon Gwinn https://github.com/gwinndr/MusicTransformer-Pytorch

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (Setup Environment)

In [ ]:
#@title nvidia-smi gpu check
!nvidia-smi

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/MIDI-Tempo-Detective
!pip install torch
!pip install tqdm
!pip install matplotlib

!pip install torch-summary
!pip install sklearn

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import tqdm
from tqdm import tqdm
import random
import secrets
from collections import OrderedDict

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

print('Loading TMIDIX and GPT2RGAX modules...')
os.chdir('/content/MIDI-Tempo-Detective')
import TMIDIX
from GPT2RGAX import *

import matplotlib.pyplot as plt
from torchsummary import summary
from sklearn import metrics

os.chdir('/content/')

# (FROM SCRATCH) Download and process MIDI dataset

In [ ]:
#@title Download original LAKH MIDI Dataset (Recommended)

%cd /content/Dataset/

!wget 'http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz'
!tar -xvf 'lmd_full.tar.gz'
!rm 'lmd_full.tar.gz'

%cd /content/

# (PROCESS)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

sorted_or_random_file_loading_order = False # Sorted order is NOT usually recommended
dataset_ratio = 1 # Change this if you need more data


print('TMIDIX MIDI Processor')
print('Starting up...')
###########

files_count = 0

gfiles = []

melody_chords_f = []

###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"
# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

print('Randomizing file list...')
random.shuffle(filez)

TMIDIX.Tegridy_Any_Pickle_File_Writer(filez, '/content/filez')

print('Processing MIDI files. Please wait...')
for f in tqdm(filez[:int(len(filez) * dataset_ratio)]):
    try:
        fn = os.path.basename(f)
        fn1 = fn.split('.')[0]

        #print('Loading MIDI file...')
        score = TMIDIX.midi2score(open(f, 'rb').read())

        events_matrix = []

        itrack = 1

        while itrack < len(score):
            for event in score[itrack]:         
                if event[0] == 'note' or event[0] == 'set_tempo':
                    events_matrix.append(event)
            itrack += 1

        events_matrix.sort(key=lambda x: x[1])

        tempos = []


        for e in events_matrix:
          if e[0] != 'set_tempo':
            tempos.append(e[1])
          else:
            if len(tempos) > 15 and len(tempos) < 512:
              melody_chords_f.append([score[0], tempos[0][1:], tempos[1:]])
            tempos = []
            tempos.append(e)
  

        files_count += 1
        
    except KeyboardInterrupt:
        print('Saving current progress and quitting...')
        break  

    except:
        print('Bad MIDI:', f)
        continue

print('=' * 70)
TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/MIDI-Tempo-Detective-DATA')        
print('Done!')   
print('=' * 70)

print('Resulting Stats:')
print('=' * 70)
print('Total MIDI Files:', files_count)
print('=' * 70)

# (PREP INTs)

In [ ]:
#@title Process and prep INTs...

print('=' * 70)
print('Processing the dataset...')

DATA = []

for m in melody_chords_f:
  if len(m[2]) < 256 and len(m[2]) > 15:
    DATA.append(m)

FINAL = []

for D in tqdm(DATA):
  INTS = []
  INTS.append(259) # SOS/EOS

  ticks1, ticks2, ticks3 = min(256*256*255, D[0]).to_bytes(3, 'big')
  INTS.extend([ticks1, ticks2, ticks3])

  INTS.append(257) # TICKS PAD

  pe = D[2][0]
  for d in D[2][1:]:
    dtime = min(256*255, d - pe)
    dt1, dt2 = dtime.to_bytes(2, 'big')
    INTS.extend([dt1, dt2])

    pe = d

  INTS.extend([258]) # TIMES PAD

  tempo1, tempo2, tempo3 = min(256*256*255, D[1][1]).to_bytes(3, 'big')
  INTS.extend([tempo1, tempo2, tempo3])

  INTS += [259] * (256 - len(INTS)) 

  #INTS.append(259) # SOS/EOS
  if len(INTS) == 256: 
    FINAL.extend(INTS)

print('Done!')        
print('=' * 70)

TMIDIX.Tegridy_Any_Pickle_File_Writer(FINAL, '/content/MIDI-Tempo-Detective-INTs')
print('=' * 70)

train_data1 = FINAL

print('Total INTs:', len(train_data1))
print('Minimum INT:', min(train_data1))
print('Maximum INT:', max(train_data1))
print('Unique INTs:', len(set(train_data1)))
print('Intro/Zero INTs:', train_data1.count(0))
print('=' * 70)

# (LOAD INTs)

In [ ]:
#@title Load processed INTs dataset

SEQ_LEN = max_seq

BATCH_SIZE = 32 # Change this to your specs

# DO NOT FORGET TO ADJUST MODEL PARAMS IN GPT2RGAX module to your specs

print('=' * 50)
print('Loading training data...')

data_train, data_val = torch.Tensor(train_data1), torch.Tensor(train_data1)

class MusicSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):

        idx = secrets.randbelow((self.data.size(0) // (self.seq_len))-1) * (self.seq_len)

        x = self.data[idx: idx + self.seq_len].long()
        trg = self.data[(idx+1): (idx+1) + self.seq_len].long()
        
        return x, trg

    def __len__(self):
        return (self.data.size(0) // self.seq_len)-1

train_dataset = MusicSamplerDataset(data_train, SEQ_LEN)
val_dataset   = MusicSamplerDataset(data_val, SEQ_LEN)
train_loader  = DataLoader(train_dataset, batch_size = BATCH_SIZE)
val_loader    = DataLoader(val_dataset, batch_size = BATCH_SIZE)

print('=' * 50)
print('Total INTs in the dataset', len(train_data1))
print('Total unique INTs in the dataset', len(set(train_data1)))
print('Max INT in the dataset', max(train_data1))
print('Min INT in the dataset', min(train_data1))
print('=' * 50)
print('Length of the dataset:',len(train_dataset))
print('Number of batched samples per epoch:', len(train_data1) // max_seq // BATCH_SIZE)
print('=' * 50)
print('Sample train dataset:', train_dataset[0])
print('Sample val dataset:', val_dataset[0])
print('=' * 50)
print('Train loader length:', len(train_loader))
print('Val loader length:', len(val_loader))
print('=' * 50)
print('Done! Enjoy! :)')
print('=' * 50)

# (TRAIN)

# Train the model

In [ ]:
#@title Train

DIC_SIZE = 260

# DO NOT FORGET TO ADJUST MODEL PARAMS IN GPT2RGAX module to your specs

config = GPTConfig(DIC_SIZE, 
                   max_seq,
                   dim_feedforward=256,
                   n_layer=32, 
                   n_head=16, 
                   n_embd=256,
                   enable_rpr=True,
                   er_len=max_seq)

# DO NOT FORGET TO ADJUST MODEL PARAMS IN GPT2RGAX module to your specs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT(config)

model = nn.DataParallel(model)

model.to(device)

#=====

init_step = 0
lr = LR_DEFAULT_START
lr_stepper = LrStepTracker(d_model, SCHEDULER_WARMUP_STEPS, init_step)
eval_loss_func = nn.CrossEntropyLoss(ignore_index=DIC_SIZE)
train_loss_func = eval_loss_func

opt = Adam(model.parameters(), lr=lr, betas=(ADAM_BETA_1, ADAM_BETA_2), eps=ADAM_EPSILON)
lr_scheduler = LambdaLR(opt, lr_stepper.step)


#===

best_eval_acc        = 0.0
best_eval_acc_epoch  = -1
best_eval_loss       = float("inf")
best_eval_loss_epoch = -1
best_acc_file = '/content/gpt2_rpr_acc.pth'
best_loss_file = '/content/gpt2_rpr_loss.pth'
loss_train, loss_val, acc_val = [], [], []

for epoch in range(0, epochs):
    new_best = False
    
    loss = train(epoch+1, 
                 model, train_loader, 
                 train_loss_func, 
                 opt, 
                 lr_scheduler, 
                 num_iters=-1, 
                 save_checkpoint_steps=4000)
    
    loss_train.append(loss)
    
    eval_loss, eval_acc = eval_model(model, val_loader, eval_loss_func, num_iters=-1)
    loss_val.append(eval_loss)
    acc_val.append(eval_acc)
    
    if(eval_acc > best_eval_acc):
        best_eval_acc = eval_acc
        best_eval_acc_epoch  = epoch+1
        torch.save(model.state_dict(), best_acc_file)
        new_best = True

    if(eval_loss < best_eval_loss):
        best_eval_loss       = eval_loss
        best_eval_loss_epoch = epoch+1
        torch.save(model.state_dict(), best_loss_file)
        new_best = True
    
    if(new_best):
        print("Best eval acc epoch:", best_eval_acc_epoch)
        print("Best eval acc:", best_eval_acc)
        print("")
        print("Best eval loss epoch:", best_eval_loss_epoch)
        print("Best eval loss:", best_eval_loss)

In [ ]:
#@title Eval funct to eval separately if needed


#=====

init_step = 0
lr = LR_DEFAULT_START
lr_stepper = LrStepTracker(d_model, SCHEDULER_WARMUP_STEPS, init_step)
eval_loss_func = nn.CrossEntropyLoss(ignore_index=DIC_SIZE)
train_loss_func = eval_loss_func

opt = Adam(model.parameters(), lr=lr, betas=(ADAM_BETA_1, ADAM_BETA_2), eps=ADAM_EPSILON)
lr_scheduler = LambdaLR(opt, lr_stepper.step)


eval_loss, eval_acc = eval_model(model, val_loader, eval_loss_func, num_iters=-1)

# (SAVE)

In [ ]:
#@title Save the model

print('Saving the model...')
full_path_to_model_checkpoint = "/content/MIDI-Tempo-Detective-Trained-Model.pth" #@param {type:"string"}
torch.save(model.state_dict(), full_path_to_model_checkpoint)
print('Done!')

# Congrats! You did it! :)